<a href="https://colab.research.google.com/github/kiteday/GlassDocter/blob/main/glass_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


0.1 라이브러리 임포트



In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np
import pandas as pd
import os

**0.2 하이퍼파라미터 세팅**

In [ ]:
batch_size = 128
num_epochs = 30

word_vec_size = 256
dropout_p = 0.3

hidden_size = 512
num_layers = 4

learning_rate = 0.001

In [ ]:
#device = torch.device('cuda')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. train, test dataset 가져오기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/NLP

/content/gdrive/MyDrive/NLP


In [ ]:
!pip install import_ipynb
import import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=2837806f1f2a358c23b6df9b809b6ab8e4822f6f1f2bd7bb45123603754448ed
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [ ]:
from data_loader import DataLoader

importing Jupyter notebook from data_loader.ipynb
1.10.0+cu111
     |████████████████████████████████| 53 kB 1.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [ ]:
loaders = DataLoader(
    train_fn='/content/gdrive/MyDrive/NLP/project/data/GD_train.tsv',
    batch_size=batch_size,
    vaild_ratio=.2,
    device=-1,
    max_vocab=999999,
    min_freq=5,
)

In [ ]:
test_loaders = DataLoader(
    train_fn='/content/gdrive/MyDrive/NLP/project/data/GD_test.tsv',
    batch_size=batch_size,
    vaild_ratio=.01,
    device=-1,
    max_vocab=999999,
    min_freq=5,
)

# 2. 대략적인 데이터 형태

In [ ]:
print("|train| =", len(loaders.train_loader.dataset),
      "|valid| =", len(loaders.vaild_loader.dataset))
vocab_size = len(loaders.text.vocab)
num_classes = len(loaders.label.vocab)
print("|vocab| =", vocab_size, '|classes| =', num_classes)

|train| = 739 |valid| = 185
|vocab| = 92 |classes| = 7


# 3. 데이터 로드함수
학습시킬 때 batch_size 단위로 끊어 로드하기 위함

In [ ]:
n=3
for i, data in enumerate(loaders.train_loader):
  labels = data.label
  texts = data.text

  if i>n:
    break
  print("[%d]" %i)
  print("한 번에 로드되는 데이터 크기:", len(labels))

  #출력
  for j in range(n):
    label = labels[j].numpy()
    text = texts[j].numpy()
    print("label: ",label)
    print("text: ", text.shape)
    

[0]
한 번에 로드되는 데이터 크기: 128
label:  5
text:  (9,)
label:  5
text:  (9,)
label:  1
text:  (9,)
[1]
한 번에 로드되는 데이터 크기: 128
label:  0
text:  (4,)
label:  2
text:  (4,)
label:  0
text:  (4,)
[2]
한 번에 로드되는 데이터 크기: 128
label:  2
text:  (2,)
label:  3
text:  (2,)
label:  5
text:  (2,)
[3]
한 번에 로드되는 데이터 크기: 99
label:  0
text:  (27,)
label:  2
text:  (27,)
label:  2
text:  (27,)


# 4. 모델선언

In [ ]:
class RNN(nn.Module):
  def __init__(self,
               input_size,
               word_vec_size,
               hidden_size,
               n_classes,
               num_layers=4,
               dropout_p=0.3
               ):
    super(RNN, self).__init__()

    self.input_size = input_size
    self.word_vec_size = word_vec_size
    self.hidden_size = hidden_size
    self.n_classes = n_classes
    self.num_layers = num_layers
    self.dropout_p = dropout_p

    self.emb = nn.Embedding(input_size, word_vec_size)

    self.lstm = nn.LSTM(input_size=word_vec_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        dropout=dropout_p,
                        batch_first=True,
                        bidirectional=True)
    self.fc = nn.Linear(hidden_size*2, num_classes)

    self.activation = nn.LogSoftmax(dim=-1)

  def forward(self, x):
    x=self.emb(x)

    x, _ =self.lstm(x)

    out = self.activation(self.fc(x[:,-1]))

    return out

In [ ]:
model = RNN(input_size=vocab_size,
             word_vec_size=word_vec_size,
             hidden_size=hidden_size,
             n_classes=num_classes,
             num_layers=num_layers,
             dropout_p=dropout_p)

In [ ]:
def ComputeArr(dloader, imodel):
  correct = 0
  total = 0

  model.eval()    
  for i,data in enumerate(dloader):
    texts = data.text.to(device)
    labels = data.label.to(device)

    output = model(texts)
    _, output_index = torch.max(output, 1)
        
    total += labels.size(0)
    correct += (output_index == labels).sum().float()
  
  model.train()      
  return (100*correct/total).cpu().numpy()

In [ ]:
print("Accuracy of Test Data: %.2f" % ComputeArr(loaders.vaild_loader, model.cuda()))

Accuracy of Test Data: 15.14


# 5. loss, optimizer

In [ ]:
loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 6. 학습

In [ ]:
total_step = len(loaders.train_loader)
for epoch in range(num_epochs):
  for i, data in enumerate(loaders.train_loader):
    texts = data.text.to(device)
    labels = data.label.to(device)

    print("[%d]" %i)
    
    outputs = model(texts)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 7 ==0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accr:{:.2f}'
      .format(epoch+1, num_epochs, i+1, total_step,
              loss.item(),
              ComputeArr(loaders.vaild_loader, model)))

[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]
[0]
[1]
[2]
[3]
[4]
[5]


In [ ]:
print("Accuracy of Valid Data: %.2f" %ComputeArr(loaders.vaild_loader, model))

Accuracy of Valid Data: 92.43


## *학습데이터 저장

In [ ]:
netname = '/content/gdrive/MyDrive/NLP/project/data/rnn_glass5.pkl'
torch.save(model, netname, )

## *학습된 모델 로드

In [ ]:
netname = '/content/gdrive/MyDrive/NLP/project/data/rnn_glass5.pkl'
model = torch.load(netname)

# 7. 테스트

In [ ]:
print("Accuracy of Valid Data: %.2f" %ComputeArr(loaders.vaild_loader, model))

Accuracy of Valid Data: 92.43


In [ ]:
def Testing(dloader, imodel):
    correct=0
    total=0
    
    model.eval() #test mode
    for i,data in enumerate(dloader): #batch_size 만큼
        texts=data.text.to(device)
        labels=data.label.to(device)
        
        # Forward prop.
        output=model(texts) #(batch_size, num_classes)
        _, output_index=torch.max(output,1) # (batch_size, 1)
        
        total+=labels.size(0)
        correct+=(output_index==labels).sum().float()
        #print(output_index)
    model.train()
    print(len(output_index))
    if (i==1):
      for i in range(3):
        #print(texts[i])
        if(output_index[i]==labels[i]) :
          print(output_index[i],'\t',labels[i], 'Correct!')
        else :
          print(output_index[i],'\t',labels[i], 'Wrong...')
    print('predicted label is : ',str(output_index[0])[7])
    get_index=str(output_index[0])[7]
    

    if get_index == '3':
      print("얼굴 묘사 없음")
    elif get_index == '0':
      print("둥근형은 각진 안경테가 어울립니다.")
    elif get_index == '1':
      print("계란형 얼굴은 모든 안경테가 어울립니다.")
    elif get_index == '2':
      print("각진형은 둥근 안경테가 어울립니다.")
    elif get_index == '4':
      print("긴 형은 옆으로 퍼진 안경테가 어울립니다.")
    elif get_index == '5':
      print("역삼각형은 네모난 안경테가 어울립니다.")
    elif get_index == '6':
      print("삼각형은 아래가 넓은 안경테가 어울립니다.")

    #print(output_index,'\t',labels)
    return (100*correct/total).cpu().numpy() # tensor -> numpy 

In [ ]:
end = 1
while end==1:
  sentence = input("얼굴을 묘사해주세요 : ")+' '
  if sentence == '0' :
    break
  text = pd.read_csv('/content/gdrive/MyDrive/NLP/project/data/GD_train.tsv', delimiter = '\t')
  #print(text)
  #print(text.loc[80:84])
  text.loc[873]=('N','face shape ' +sentence+ 'shape wide straight forehead overall face color warm natural color redness face shape '+sentence+'shape wide straight forehead overall face color warm natural color redness beard philtrum jawline length fur short shape rap industry standard ')
  text.to_csv('/content/gdrive/MyDrive/NLP/project/data/GD_train.tsv', 
                  header=True, index=False, sep='\t')

  loaders = DataLoader(
      train_fn='/content/gdrive/MyDrive/NLP/project/data/GD_train.tsv',
      batch_size=batch_size,
      #vaild_ratio,
      device=0,
      max_vocab=999999,
      min_freq=5,
  )

  Testing(loaders.vaild_loader, model)


얼굴을 묘사해주세요 : egg
57
tensor(1, device='cuda:0') 	 tensor(3, device='cuda:0') Wrong...
tensor(5, device='cuda:0') 	 tensor(3, device='cuda:0') Wrong...
tensor(5, device='cuda:0') 	 tensor(5, device='cuda:0') Correct!
predicted label is :  1
계란형 얼굴은 모든 안경테가 어울립니다.
얼굴을 묘사해주세요 : round
57
tensor(0, device='cuda:0') 	 tensor(3, device='cuda:0') Wrong...
tensor(5, device='cuda:0') 	 tensor(3, device='cuda:0') Wrong...
tensor(5, device='cuda:0') 	 tensor(5, device='cuda:0') Correct!
predicted label is :  0
둥근형은 각진 안경테가 어울립니다.
얼굴을 묘사해주세요 : square
57
tensor(2, device='cuda:0') 	 tensor(3, device='cuda:0') Wrong...
tensor(5, device='cuda:0') 	 tensor(3, device='cuda:0') Wrong...
tensor(5, device='cuda:0') 	 tensor(5, device='cuda:0') Correct!
predicted label is :  2
각진형은 둥근 안경테가 어울립니다.
얼굴을 묘사해주세요 : long
57
tensor(4, device='cuda:0') 	 tensor(3, device='cuda:0') Wrong...
tensor(5, device='cuda:0') 	 tensor(3, device='cuda:0') Wrong...
tensor(5, device='cuda:0') 	 tensor(5, device='cuda:0') Correct!

KeyboardInterrupt: ignored